## Imports and Setup

In [1]:
import os
import sys
import subprocess
sys.path.insert(0, "../utils/")
import predict
import config

import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm

pd.set_option('mode.chained_assignment', None)

%load_ext autoreload
%autoreload 2

## Load Dataset

In [8]:
name = 'Saint Peter'
bldgs_file = f'../data/vectors/building_footprints_annotated_DOM.gpkg'

bldgs = gpd.read_file(bldgs_file)
bldgs = bldgs[bldgs.shape_name == name]
print(f"Data dimensions: {bldgs.shape}")
bldgs.head(3)

Data dimensions: (1272, 8)


,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,NaN,NaN,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,NaN,NaN,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,NaN,NaN,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


## Generate Post-disaster Predictions

In [9]:
in_file = '../data/rasters/Ortho/ortho_DOM.tif'
exp_config = '../configs/config_01.yaml'
c = config.create_config(exp_config)
exp_name = c['exp_name']
exp_dir = f'../exp/{exp_name}/'

classes = ['BLUE_TARP', 'CONCRETE_CEMENT', 'HEALTHY_METAL', 'INCOMPLETE', 'IRREGULAR_METAL']
model = predict.load_model(c['model'], exp_dir, n_classes=len(classes), dropout=c['dropout'])
bldgs = predict.generate_predictions(
    bldgs, model, in_file, exp_dir, class_name='roof_material', classes=classes, scale=1.5
)   
bldgs.head(3)

Model file ../exp/exp-01/best_model.pth successfully loaded.


  0%|          | 0/1272 [00:00<?, ?it/s]

,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,NaN,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,NaN,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,NaN,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [10]:
in_file = '../data/rasters/Ortho/ortho_DOM.tif'
exp_config = '../configs/config_02.yaml'
c = config.create_config(exp_config)
exp_name = c['exp_name']
exp_dir = f'../exp/{exp_name}/'

classes = ['FLAT', 'GABLE', 'HIP', 'NO_ROOF']
model = predict.load_model(c['model'], exp_dir, n_classes=len(classes), dropout=c['dropout'])
bldgs = predict.generate_predictions(
    bldgs, model, in_file, exp_dir, class_name='roof_type', classes=classes, scale=1.5
)
bldgs.head(3)

Model file ../exp/exp-02/best_model.pth successfully loaded.


  0%|          | 0/1272 [00:00<?, ?it/s]

,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,HIP,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,GABLE,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,GABLE,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [11]:
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}_post.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Generate Pre-disaster Predictions

In [12]:
in_file = '../data/rasters/Drone/drone_colihaut_DOM.tif'
exp_config = '../configs/config_01.yaml'
c = config.create_config(exp_config)
exp_name = c['exp_name']
exp_dir = f'../exp/{exp_name}/'

classes = ['BLUE_TARP', 'CONCRETE', 'HEALTHY_METAL', 'INCOMPLETE', 'IRREGULAR_METAL']
model = predict.load_model(c['model'], exp_dir, n_classes=len(classes), dropout=c['dropout'])
bldgs = predict.generate_predictions(
    bldgs, model, in_file, exp_dir, class_name='roof_material', classes=classes, scale=1.5
)   
bldgs.head(3)

Model file ../exp/exp-01/best_model.pth successfully loaded.


  0%|          | 0/1272 [00:00<?, ?it/s]

,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,HIP,IRREGULAR_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,GABLE,IRREGULAR_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,GABLE,IRREGULAR_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [13]:
exp_config = '../configs/config_02.yaml'
c = config.create_config(exp_config)
exp_name = c['exp_name']
exp_dir = f'../exp/{exp_name}/'

classes = ['FLAT', 'GABLE', 'HIP', 'NO_ROOF']
model = predict.load_model(c['model'], exp_dir, n_classes=len(classes), dropout=c['dropout'])
bldgs = predict.generate_predictions(
    bldgs, model, in_file, exp_dir, class_name='roof_type', classes=classes, scale=1.5
)
bldgs.head(3)

Model file ../exp/exp-02/best_model.pth successfully loaded.


  0%|          | 0/1272 [00:00<?, ?it/s]

,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,HIP,IRREGULAR_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,HIP,IRREGULAR_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,HIP,IRREGULAR_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [14]:
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}_pre.gpkg')
bldgs.to_file(out_file, driver='GPKG')